In [1]:
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings('ignore')
from sklearn.base import BaseEstimator, clone
from sklearn.base import TransformerMixin
import pickle
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer

In [2]:
X_test = pd.read_csv('input/test.tsv', delimiter='\t')

In [3]:
class columnDropperTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def transform(self, X, y=None):
        return X.drop(self.columns, axis=1)

    def fit(self, X=None, y=None):
        return self


class CustomDomainTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.domains = None

    def fit(self, X=None, y=None):
        return self

    def transform(self, X=None, y=None):
        import tldextract
        X['domain'] = X['url'].apply(
            lambda url: tldextract.extract(url).domain)
        X['subdomain'] = X['url'].apply(
            lambda url: tldextract.extract(url).subdomain)
        X['suffix'] = X['url'].apply(
            lambda url: tldextract.extract(url).suffix)
        X.drop('url', inplace=True, axis=1)
        return X


class CustomBoilerPlateTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X=None, y=None):
        X['boilerplate'] = X['boilerplate'].replace(to_replace=':null',
                                                    value=':""',
                                                    regex=True)
        X.boilerplate = X.boilerplate.map(eval)
        X = pd.concat(
            [X.drop('boilerplate', axis=1), X['boilerplate'].apply(pd.Series)],
            axis=1)
        X.drop('url', inplace=True, axis=1)
        return X


def prepare_additional(X):
    X.replace(to_replace='?', value=np.nan, inplace=True)
    X['alchemy_category_score'] = X['alchemy_category_score'].astype(
        dtype='float')
    X['news_front_page'] = X['news_front_page'].astype(dtype='float')
    X.alchemy_category[X.groupby('alchemy_category')['alchemy_category'].
                       transform('count').lt(100)] = 'RARE'
    return X


class CustomImputer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.missing_map = {}

    def fit(self, X, y=None):
        num_atts = X.select_dtypes(exclude='object').columns.tolist()

        for att in num_atts:
            self.missing_map.update({att: X[att].median()})

        cat_atts = X.select_dtypes(include='object').columns.tolist()

        for att in cat_atts:
            self.missing_map.update({att: X[att].value_counts().index[0]})

        return (self)

    def transform(self, X, y=None):

        for att, impute_value in self.missing_map.items():
            X[att] = X[att].fillna(impute_value)
        return (X)

In [4]:
with open('output/enc.pcl', 'rb') as picklefile:
    enc = pickle.load(picklefile)

with open('output/model.pcl', 'rb') as picklefile:
    model = pickle.load(picklefile)

with open('output/cv_body.pcl', 'rb') as picklefile:
    cv_body = pickle.load(picklefile)

with open('output/cv_title.pcl', 'rb') as picklefile:
    cv_title = pickle.load(picklefile)

with open('output/scaler.pcl', 'rb') as picklefile:
    scaler = pickle.load(picklefile)

with open('output/custom_imputer.pcl', 'rb') as picklefile:
    imputer = pickle.load(picklefile)

with open('output/numerical_cols.pcl', 'rb') as picklefile:
    numerical_cols = pickle.load(picklefile)

with open('output/categorical_cols.pcl', 'rb') as picklefile:
    categorical_cols = pickle.load(picklefile)

In [5]:
X_test = columnDropperTransformer(['urlid', 'is_news',
                                   'framebased']).fit_transform(X_test)
X_test = CustomDomainTransformer().fit_transform(X_test)
X_test = CustomBoilerPlateTransformer().fit_transform(X_test)
X_test = columnDropperTransformer(['related']).fit_transform(X_test)
X_test = FunctionTransformer(prepare_additional).fit_transform(X_test)
X_test.replace(to_replace='?', value=np.nan, inplace=True)
X_test = imputer.transform(X_test)

In [6]:
X_new = pd.DataFrame(enc.transform(X_test[categorical_cols]).toarray())
X_new.columns = enc.get_feature_names(categorical_cols)
titles_test = X_test.title
bodies_test = X_test.body
X_test.drop(['title', 'body'], inplace=True, axis=1)
X_test = pd.concat([X_test[numerical_cols], X_new], axis=1)

In [7]:
transformed_titles_test = cv_title.transform(titles_test)
df_cv = pd.DataFrame(transformed_titles_test.todense(),
                     columns=cv_title.get_feature_names())
X_test = pd.concat([X_test, df_cv], axis=1)
transformed_bodies_test = cv_body.transform(bodies_test)
df_cv = pd.DataFrame(transformed_bodies_test.todense(),
                     columns=cv_body.get_feature_names())
X_test = pd.concat([X_test, df_cv], axis=1)
X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])
X_test = X_test.loc[:, ~X_test.columns.duplicated()]

In [8]:
model.predict(X_test)

array([1, 0, 0, ..., 1, 1, 1], dtype=int64)

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer


numerical_prep = Pipeline(steps=[
                                    ('imputer',imputer),
                                    ('scaler',scaler)
])

 
categorical_prep = Pipeline(steps=[
                                   ('domain', custom_domain_transformer),
                                    ('delete_cols',column_dropper_transformer), 
                                    
                                     ('boilerplate',custom_boilerplate_transformer),# DOPUNA
                                    ('additional',function_prepare_additional),
                                    ('imputer',imputer),
                                    ('onehot',enc),
                                     
                                    ]
                                  )

 
preprocessor = ColumnTransformer(transformers=[
                                     ('num',numerical_prep,numerical_cols),
                                     ('cat',categorical_prep,categorical_cols)
    
                                 ]
                                )
 
my_pipeline =Pipeline(steps = [
                         ('preprocessor',preprocessor),
                        ('cv_title',cv_title),
                         ('cv_body',cv_body),
                         ('model',model)
                         ]
                     )
my_pipeline.transform(X_test)